In [ ]:
import requests
import json
import pandas as pd

In [ ]:
df = pd.read_excel('Observed properties.xlsx')
df_csv_uom = df.to_csv('Observed properties.csv', index=False)
df_csv_uom = pd.read_csv('Observed properties.csv')

In [ ]:
df_csv_obs = pd.read_csv('Creazione_obs_property_....csv', sep=';')
df_csv_obs
#Observed property rispetto alla city o alla tipologia

In [ ]:
df = pd.read_excel('percorso_file.xlsx', sheet_name='Things')
df_csv = df.to_csv('percorso_file.csv', index=False)
df_csv = pd.read_csv('percorso_file.csv')

In [ ]:
#BATCH
json_struct = []
datastream_counter = #da valorizzare
fixed_part = {
    "POST to": "v1.1/$batch",
    "Headers": "Content-Type: application/json",
    "requests": []  
}
json_struct.append(fixed_part)

In [ ]:
merged_df = pd.merge(df_csv_obs, df_csv_uom, left_on='description', right_on='name ', how='inner')
merged_df

In [ ]:
added_datastreams = []

for i in range(1, 12): #change the range
    #Lettura things precedentemente create
    with open(f"json\\thing_{i}.json", "r") as json_file:
        thing_data = json.load(json_file)
        thing_id = thing_data["body"]["@iot.id"]

    for idx, loc_row in merged_df.iterrows():
        iot_id_gen = f"DST.FE.{datastream_counter:03}"
        json_data = {
            "id": str(iot_id_gen),
            "method": "post",
            "url": "Datastreams",
            "body": {
                
                "Thing": {
                    "@iot.id": thing_id
                },
                "Sensor": {
                    "@iot.id": "SEN." # number of sensors
                 },
                "ObservedProperty": {
                    "@iot.id": loc_row["id "]
                },
                "name": f"{loc_row['name']} ({loc_row['description_x']})@{thing_id}",
                "description": loc_row["description_x"],
                "unitOfMeasurement": {
                    "name": loc_row["description uom"],
                    "symbol": loc_row["uom"],
                    "definition": loc_row["definition_x"]
                },
                "observationType": "OM_Measurement",
                "properties": {
                    "identifier": str(iot_id_gen)
                }
            }
        }
        
        if json_data not in added_datastreams:
            json_struct[0]["requests"].append(json_data)
            added_datastreams.append(json_data)
        
        datastream_counter += 1

print(f"Aggiunti {datastream_counter - 1} datastreams a json_struct.")


In [ ]:
import json

formatted_json = json.dumps(json_struct, indent=3, ensure_ascii=False)

try:
    json.loads(formatted_json)
    print("Il JSON è stato formattato correttamente.")
except json.JSONDecodeError as e:
    print(f"Errore nella formattazione: {e}")
    
with open('Datastreams.json', 'w', encoding='utf-8') as json_file:
    json_file.write(formatted_json)
